In [1]:
# Import the libraries
import pandas as pd
from datetime import datetime
import numpy as np
#pip install requests pandas
#pip install folium

In [2]:
# Import the data
df = pd.read_csv('kc_house_data.csv')
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [3]:
# Display summary statistics for the DataFrame
df.describe()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,2.161300e+04,2.161300e+04,21613.000000,21613.000000,21613.000000,2.161300e+04,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000
mean,4.580302e+09,5.400881e+05,3.370842,2.114757,2079.899736,1.510697e+04,1.494309,0.007542,0.234303,3.409430,7.656873,1788.390691,291.509045,1971.005136,84.402258,98077.939805,47.560053,-122.213896,1986.552492,12768.455652
std,2.876566e+09,3.671272e+05,0.930062,0.770163,918.440897,4.142051e+04,0.539989,0.086517,0.766318,0.650743,1.175459,828.090978,442.575043,29.373411,401.679240,53.505026,0.138564,0.140828,685.391304,27304.179631
min,1.000102e+06,7.500000e+04,0.000000,0.000000,290.000000,5.200000e+02,1.000000,0.000000,0.000000,1.000000,1.000000,290.000000,0.000000,1900.000000,0.000000,98001.000000,47.155900,-122.519000,399.000000,651.000000
25%,2.123049e+09,3.219500e+05,3.000000,1.750000,1427.000000,5.040000e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1190.000000,0.000000,1951.000000,0.000000,98033.000000,47.471000,-122.328000,1490.000000,5100.000000
50%,3.904930e+09,4.500000e+05,3.000000,2.250000,1910.000000,7.618000e+03,1.500000,0.000000,0.000000,3.000000,7.000000,1560.000000,0.000000,1975.000000,0.000000,98065.000000,47.571800,-122.230000,1840.000000,7620.000000
75%,7.308900e+09,6.450000e+05,4.000000,2.500000,2550.000000,1.068800e+04,2.000000,0.000000,0.000000,4.000000,8.000000,2210.000000,560.000000,1997.000000,0.000000,98118.000000,47.678000,-122.125000,2360.000000,10083.000000
max,9.900000e+09,7.700000e+06,33.000000,8.000000,13540.000000,1.651359e+06,3.500000,1.000000,4.000000,5.000000,13.000000,9410.000000,4820.000000,2015.000000,2015.000000,98199.000000,47.777600,-121.315000,6210.000000,871200.000000


In [4]:
# Display data types for each column
print(df.dtypes)

id                 int64
date              object
price            float64
bedrooms           int64
bathrooms        float64
sqft_living        int64
sqft_lot           int64
floors           float64
waterfront         int64
view               int64
condition          int64
grade              int64
sqft_above         int64
sqft_basement      int64
yr_built           int64
yr_renovated       int64
zipcode            int64
lat              float64
long             float64
sqft_living15      int64
sqft_lot15         int64
dtype: object


In [5]:
# Detect missing values
na_counts = df.isna().sum()

# Display the count of missing values for each column
print(na_counts)

id               0
date             0
price            0
bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
grade            0
sqft_above       0
sqft_basement    0
yr_built         0
yr_renovated     0
zipcode          0
lat              0
long             0
sqft_living15    0
sqft_lot15       0
dtype: int64


In [6]:
# Remove Y and features not needed
X=df.copy().drop(columns='price')
X.head()

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [7]:
# FEATURE ENGINEERING -  combining lat and long so they can be viewed as one characteristic.
# Combine latitude and longitude into a single feature
X['location'] = list(zip(df.lat, df.long))

In [8]:
# FEATURE ENGINEERING -  measure the distance from a home's location to the County/City's largest employers, Amazon, Starbucks, and Nordstums

# This computes probable commute time.


from math import radians, sin, cos, sqrt, atan2

# Function to calculate distance using Haversine formula
def haversine(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    r = 6371  # Radius of Earth in kilometers
    return r * c

# Define city center coordinates (example: downtown coordinates)
city_center_lat = 47.615257
city_center_lon = -122.338356

# Calculate distance to city center for each row
X['distance_to_Seattle'] = X.apply(lambda row: haversine(row['lat'], row['long'], city_center_lat, city_center_lon), axis=1)

# Display the DataFrame with the new distance_to_center feature
print(X[['lat', 'long', 'distance_to_Seattle']])

           lat     long  distance_to_Seattle
0      47.5112 -122.257            13.082094
1      47.7210 -122.319            11.847081
2      47.7379 -122.233            15.754191
3      47.5208 -122.393            11.274888
4      47.6168 -122.045            21.989458
...        ...      ...                  ...
21608  47.6993 -122.346             9.362676
21609  47.5107 -122.362            11.760782
21610  47.5944 -122.299             3.752959
21611  47.5345 -122.069            22.111252
21612  47.5941 -122.299             3.773671

[21613 rows x 3 columns]


In [9]:
# FEATURE ENGINEERING -  measure the distance from a home's location to MSFT HQ
# This computes probable commute time.


# Function to calculate distance using Haversine formula
def haversine(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    r = 6371  # Radius of Earth in kilometers
    return r * c

# Define city center coordinates (example: downtown coordinates)
MSFT_lat = 47.643543
MSFT_lon = -122.130821

# Calculate distance to city center for each row
X['distance_to_MSFT'] = X.apply(lambda row: haversine(row['lat'], row['long'], MSFT_lat, MSFT_lon), axis=1)

# Display the DataFrame with the new distance_to_center feature
print(X[['lat', 'long', 'distance_to_MSFT']])

           lat     long  distance_to_MSFT
0      47.5112 -122.257         17.496867
1      47.7210 -122.319         16.511564
2      47.7379 -122.233         12.983608
3      47.5208 -122.393         23.936895
4      47.6168 -122.045          7.085291
...        ...      ...               ...
21608  47.6993 -122.346         17.263568
21609  47.5107 -122.362         22.779616
21610  47.5944 -122.299         13.738786
21611  47.5345 -122.069         12.981182
21612  47.5941 -122.299         13.752121

[21613 rows x 3 columns]


In [10]:
# FEATURE ENGINEERING -  measure the distance from a home's location to Boeing Seatle Offices
# This computes probable commute time.


# Function to calculate distance using Haversine formula
def haversine(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    r = 6371  # Radius of Earth in kilometers
    return r * c

# Define city center coordinates (example: downtown coordinates)
Boeing_lat = 47.532733
Boeing_lon = -122.316916

# Calculate distance to city center for each row
X['distance_to_Boeing'] = X.apply(lambda row: haversine(row['lat'], row['long'], Boeing_lat, Boeing_lon), axis=1)

# Display the DataFrame with the new distance_to_center feature
print(X[['lat', 'long', 'distance_to_Boeing']])

           lat     long  distance_to_Boeing
0      47.5112 -122.257            5.096588
1      47.7210 -122.319           20.934918
2      47.7379 -122.233           23.664150
3      47.5208 -122.393            5.864758
4      47.6168 -122.045           22.437740
...        ...      ...                 ...
21608  47.6993 -122.346           18.649261
21609  47.5107 -122.362            4.178909
21610  47.5944 -122.299            6.987580
21611  47.5345 -122.069           18.613097
21612  47.5941 -122.299            6.954848

[21613 rows x 3 columns]


In [11]:
# FEATURE ENGINEERING - Identify a house to a school district and how that school district ranks
# This coputes the quality of public education

# Define the school district rankings
school_districts = {
    1: 'Bellevue',
    2: 'Mercer Island',
    3: 'Lake Washington',
    4: 'Issaquah',
    5: 'Shoreline',
    6: 'Vashon Island',
    7: 'Tahoma',
    8: 'All Others'
}

# Define zip code to school district mapping
zip_to_district = {
    98004: 1, 98005: 1, 98006: 1, 98007: 1, 98008: 1,  # Bellevue
    98040: 2,                                        # Mercer Island
    98033: 3, 98034: 3, 98072: 3, 98074: 3, 98052: 3, # Lake Washington
    98027: 4, 98029: 4, 98075: 4, 98038: 4,           # Issaquah
    98155: 5, 98133: 5, 98177: 5, 98160: 5,           # Shoreline
    98070: 6,                                        # Vashon Island
    98038: 7                                         # Tahoma
}

# Assign 'All Others' (rank 8) to zip codes not explicitly listed
X['school_district_rank'] = X['zipcode'].map(zip_to_district).fillna(8)

# Display the DataFrame with the new school_district feature
print(X[['zipcode', 'school_district_rank']])

       zipcode  school_district_rank
0        98178                   8.0
1        98125                   8.0
2        98028                   8.0
3        98136                   8.0
4        98074                   3.0
...        ...                   ...
21608    98103                   8.0
21609    98146                   8.0
21610    98144                   8.0
21611    98027                   4.0
21612    98144                   8.0

[21613 rows x 2 columns]


In [12]:
# FEATURE ENGINEERING - Area Type - Identifying a home as being in an urban area vs suburban vs rural  

# Zip code classification
urban_zips = {'98101', '98102', '98103', '98104', '98105', '98106', '98107', '98108', '98109', '98112', '98115', '98116', '98117', '98118', '98119', '98121', '98122', '98125', '98126', '98133', '98134', '98136', '98144', '98146', '98148', '98154', '98155', '98158', '98160', '98161', '98164', '98166', '98168', '98174', '98177', '98178', '98188', '98198', '98199'}
suburban_zips = {'98001', '98002', '98003', '98004', '98005', '98006', '98007', '98008', '98010', '98011', '98014', '98019', '98022', '98023', '98024', '98025', '98027', '98028', '98029', '98030', '98031', '98032', '98033', '98034', '98038', '98039', '98040', '98042', '98045', '98047', '98050', '98051', '98052', '98053', '98055', '98056', '98057', '98058', '98059', '98062', '98063', '98064', '98065', '98070', '98072', '98073', '98074', '98075', '98077', '98092', '98093'}

def classify_zipcode(zipcode):
    if zipcode in urban_zips:
        return 0  # Urban
    elif zipcode in suburban_zips:
        return 1  # Suburban
    else:
        return 2  # Rural

# Apply the classification to the DataFrame
X['area_type'] = X['zipcode'].apply(lambda x: classify_zipcode(str(x)))

# Display the resulting DataFrame
print(X)

               id             date  bedrooms  bathrooms  sqft_living  \
0      7129300520  20141013T000000         3       1.00         1180   
1      6414100192  20141209T000000         3       2.25         2570   
2      5631500400  20150225T000000         2       1.00          770   
3      2487200875  20141209T000000         4       3.00         1960   
4      1954400510  20150218T000000         3       2.00         1680   
...           ...              ...       ...        ...          ...   
21608   263000018  20140521T000000         3       2.50         1530   
21609  6600060120  20150223T000000         4       2.50         2310   
21610  1523300141  20140623T000000         2       0.75         1020   
21611   291310100  20150116T000000         3       2.50         1600   
21612  1523300157  20141015T000000         2       0.75         1020   

       sqft_lot  floors  waterfront  view  condition  ...      lat     long  \
0          5650     1.0           0     0          3  ..

In [13]:
# FEATURE ENGINEERING - Deterine the housing density by zip code via accessing the US Census data.
#  Combined with 

import requests
import pandas as pd

# Your valid U.S. Census API key
api_key = 'da7546bc69f821495474c275ba74ac0c5ba32a7c'

# Base URL for the API
base_url = "https://api.census.gov/data/2020/acs/acs5"

# Parameters for the API request for housing units
params_housing = {
    "get": "B25001_001E",  # Total housing units
    "for": "zip code tabulation area:*",
    "key": api_key
}

# Parameters for the API request for population
params_population = {
    "get": "B01003_001E",  # Total population
    "for": "zip code tabulation area:*",
    "key": api_key
}

# Function to fetch data from Census API
def fetch_census_data(params):
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        try:
            return response.json()
        except ValueError:
            print("Error: Response content is not valid JSON")
            print(response.text)
            return None
    else:
        print(f"Error: Received response with status code {response.status_code}")
        print(response.text)
        return None

# Fetch housing data
housing_data = fetch_census_data(params_housing)
if housing_data:
    # Convert the housing data to a pandas DataFrame
    columns_housing = housing_data[0]
    data_housing_rows = housing_data[1:]
    df_housing = pd.DataFrame(data_housing_rows, columns=columns_housing)
    df_housing['B25001_001E'] = pd.to_numeric(df_housing['B25001_001E'])
    df_housing['zip code tabulation area'] = df_housing['zip code tabulation area'].astype(str)

# Fetch population data
population_data = fetch_census_data(params_population)
if population_data:
    # Convert the population data to a pandas DataFrame
    columns_population = population_data[0]
    data_population_rows = population_data[1:]
    df_population = pd.DataFrame(data_population_rows, columns=columns_population)
    df_population['B01003_001E'] = pd.to_numeric(df_population['B01003_001E'])
    df_population['zip code tabulation area'] = df_population['zip code tabulation area'].astype(str)

# List of zip codes in King County (source: external reliable source or predefined list)
king_county_zip_codes = [
    '98001', '98002', '98003', '98004', '98005', '98006', '98007', '98008', '98010', '98011', '98014', '98019', 
    '98022', '98023', '98024', '98025', '98027', '98028', '98029', '98030', '98031', '98032', '98033', '98034',
    '98038', '98039', '98040', '98042', '98045', '98047', '98050', '98051', '98052', '98053', '98055', '98056',
    '98057', '98058', '98059', '98062', '98063', '98064', '98065', '98070', '98072', '98073', '98074', '98075',
    '98077', '98092', '98093', '98101', '98102', '98103', '98104', '98105', '98106', '98107', '98108', '98109',
    '98112', '98115', '98116', '98117', '98118', '98119', '98121', '98122', '98125', '98126', '98133', '98134',
    '98136', '98144', '98146', '98148', '98154', '98155', '98158', '98160', '98161', '98164', '98166', '98168',
    '98174', '98177', '98178', '98188', '98198', '98199'
]

# Filter data for King County zip codes
if housing_data and population_data:
    df_housing_king = df_housing[df_housing['zip code tabulation area'].isin(king_county_zip_codes)]
    df_population_king = df_population[df_population['zip code tabulation area'].isin(king_county_zip_codes)]
    
    # Merge the two DataFrames on the zip code tabulation area
    df_merged = pd.merge(df_housing_king, df_population_king, on='zip code tabulation area')
    
    # Calculate housing density (total housing units per population)
    df_merged['housing_density'] = df_merged['B25001_001E'] / df_merged['B01003_001E']
    
    # Rank the zip codes by housing density
    #df_merged['density_rank'] = df_merged['housing_density'].rank(ascending=False)
    
    # Display the DataFrame with the housing density and rank
    print(df_merged[['zip code tabulation area', 'housing_density']])

   zip code tabulation area  housing_density
0                     98001         0.347729
1                     98002         0.421098
2                     98003         0.390656
3                     98004         0.522045
4                     98005         0.411973
..                      ...              ...
77                    98177         0.409656
78                    98178         0.385634
79                    98188         0.423691
80                    98198         0.393395
81                    98199         0.462123

[82 rows x 2 columns]


In [14]:
# Rename column to match X DataFrame for merging
df_merged.rename(columns={'zip code tabulation area': 'zipcode'}, inplace=True)
# Convert zipcode to string for merging
X['zipcode'] = X['zipcode'].astype(str)

# Merge the housing density and rank into X based on zipcode
X = pd.merge(X, df_merged, on='zipcode', how='left')
X.head()

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,sqft_lot15,location,distance_to_Seattle,distance_to_MSFT,distance_to_Boeing,school_district_rank,area_type,B25001_001E,B01003_001E,housing_density
0,7129300520,20141013T000000,3,1.00,1180,5650,1.0,0,0,3,...,5650,"(47.5112, -122.257)",13.082094,17.496867,5.096588,8.0,0,10222,26507,0.385634
1,6414100192,20141209T000000,3,2.25,2570,7242,2.0,0,0,3,...,7639,"(47.721, -122.319)",11.847081,16.511564,20.934918,8.0,0,20946,44030,0.475721
2,5631500400,20150225T000000,2,1.00,770,10000,1.0,0,0,3,...,8062,"(47.7379, -122.233)",15.754191,12.983608,23.664150,8.0,1,9666,22953,0.421121
3,2487200875,20141209T000000,4,3.00,1960,5000,1.0,0,0,5,...,5000,"(47.5208, -122.393)",11.274888,23.936895,5.864758,8.0,0,8272,17083,0.484224
4,1954400510,20150218T000000,3,2.00,1680,8080,1.0,0,0,3,...,7503,"(47.6168, -122.045)",21.989458,7.085291,22.437740,3.0,1,9779,29349,0.333197


In [16]:
# Drop features
columns_to_drop=['id','lat','long','B25001_001E','B01003_001E']
X=X.drop(columns=columns_to_drop)
X.head()

,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,zipcode,sqft_living15,sqft_lot15,location,distance_to_Seattle,distance_to_MSFT,distance_to_Boeing,school_district_rank,area_type,housing_density
0,20141013T000000,3,1.00,1180,5650,1.0,0,0,3,7,...,98178,1340,5650,"(47.5112, -122.257)",13.082094,17.496867,5.096588,8.0,0,0.385634
1,20141209T000000,3,2.25,2570,7242,2.0,0,0,3,7,...,98125,1690,7639,"(47.721, -122.319)",11.847081,16.511564,20.934918,8.0,0,0.475721
2,20150225T000000,2,1.00,770,10000,1.0,0,0,3,6,...,98028,2720,8062,"(47.7379, -122.233)",15.754191,12.983608,23.664150,8.0,1,0.421121
3,20141209T000000,4,3.00,1960,5000,1.0,0,0,5,7,...,98136,1360,5000,"(47.5208, -122.393)",11.274888,23.936895,5.864758,8.0,0,0.484224
4,20150218T000000,3,2.00,1680,8080,1.0,0,0,3,8,...,98074,1800,7503,"(47.6168, -122.045)",21.989458,7.085291,22.437740,3.0,1,0.333197


In [17]:
y=df['price'].values.reshape(-1,1)
y

array([[221900.],
       [538000.],
       [180000.],
       ...,
       [402101.],
       [400000.],
       [325000.]])